<img src="img/prefect-logo.png"/>

# Part 1: Dipping your toes into Prefect
Data workflow orchestrators are used to coordinate, monitor and observe data movement. Features that make data-pipelines fault-tolerant include:
- scheduling and triggering jobs
- adding retries
- dependency and state depedencies ('if the previous job failed')
- caching expensive tasks
- deploying flows to different environment 
- visibility into execution state of all jobs in your workflows

Stop safeguarding against failure! Instead, enjoy a single pane of glass for monitoring your code. 

Let's gain visibility into our Python scripts with Prefect. All we have to do for observability, scheduling and more is to add a single `@flow` decorator (and optionally `@task` decorators). After that, we'll also create a deployment, view it in the UI, add a schedule, and watch our code run in the Prefect UI. 

## Before beginnning
Take a look at `prefect_ml.py`. Note that the only difference from regular Python code is the addition of the Prefect `@task` and `@flow` decorators (flows are made up of tasks).

## What we will do in this demo
1. Installation of dependencies
2. We will first import a flow from prefect_ml.py.
3. We will run that flow locally.
4. We will then view our flow run in the UI. 
5. We will deploy our code so that we don't need to run it locally anymore.
6. We will add a schedule to our code. 
7. We will start a local agent that can execute our code on our schedule.

## First, we'll install Prefect and some other dependencies our script has to run

In [1]:
%%capture 
!pip install prefect==2.3.1 prefect-dask sklearn pandas
# The magic capture cmd simply suppresses the install output, which is lengthy.

## Import code from prefect_ml.py
We will import the flow called `my_first_ml_flow` from `prefect_ml.py`. The function has a `@flow` decorator on it, thus making it a `flow`. We will see what is so great about that in just moments!

In [1]:
# Importing my_first_ml_flow function w/ @flow decorator
from prefect_ml import my_first_ml_flow
print("Imported my_first_ml_flow")

Imported my_first_ml_flow


## Run your flow!

In [3]:
my_first_ml_flow()

09:43:16.896 | INFO    | prefect.engine - Created flow run 'practical-peacock' for flow 'my-first-ml-flow'
09:43:16.897 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`
09:43:17.532 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://127.0.0.1:8787/status
09:43:17.774 | INFO    | Flow run 'practical-peacock' - Created task run 'create-data-2fa802fa-0' for task 'create-data'
09:43:17.774 | INFO    | Flow run 'practical-peacock' - Executing 'create-data-2fa802fa-0' immediately...
09:43:17.821 | INFO    | Task run 'create-data-2fa802fa-0' - Finished in state Completed()
09:43:17.836 | INFO    | Flow run 'practical-peacock' - Created task run 'get-models-fa4a86ce-0' for task 'get-models'
09:43:17.836 | INFO    | Flow run 'practical-peacock' - Executing 'get-models-fa4a86ce-0' immediately...
09:43:17.881 | INFO    | Task run 'get-models-fa4a86ce-0' - Finished in state Completed()
09:43:17.897 | INFO 

,model,params,accuracy
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.752809
1,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.691011
2,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.713483
3,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.651685
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.792135


As we can see, info logs were generated about this flow run. We also have a flow run name! For example, your flow run name might be 'practical peacock`. Each flow run name is unique. But now let's kick thing up a notch.

## View your flow run in the UI
1. Copy this command: `prefect orion start`. This command will allow you to start an Orion server, enabling you to view your flow!
2. Visit the `Jupyter Home Page` tab that is already open in your browser. Click "New" in the right-hand corner and choose "Terminal". Paste the command you copied :)
2. Visit http://127.0.0.1:4200. In the Flow Runs tab, you should see your flow run name (e.g. 'practical peacock'), which you can click on view its logs, task runs, parameters, execution state, and more.

<img src="img/viewing-flow-run.png" style="width: 900px;"/>

# Part 2: Create a deployment in one command
Now that you have run a flow and viewed it in the UI, let's create a deployment. Deploying your flow means you no longer need to call the function, flow or .py file locally to run your code. With a single command, any script you write in Python can be executed regularly while enabling you to be notified of failures and observe the state of your jobs at all times.

Run the following command, which will build and apply a deployment:

In [4]:
%%bash
prefect deployment build prefect_ml:my_first_ml_flow --name "ML Flow Deployment" --apply --skip-upload

Found flow 'my-first-ml-flow'
Deployment YAML created at 
'/Users/bean/Documents/prefect-in-jupyter-notebook/my_first_ml_flow-deployment.y
aml'.
Deployment 'my-first-ml-flow/ML Flow Deployment' successfully created with id 
'3490b13d-8a13-4d85-b490-b7de3bcdf649'.

To execute flow runs from this deployment, start an agent that pulls work from 
the the 'default' work queue:
$ prefect agent start -q 'default'


We don't need to worry about starting an agent quite yet. For now, let's head on over to the UI to view our newly created deployment!

## Add a schedule to your deployment
[In the UI](http://127.0.0.1:4200), take a look at the `Deployments` tab. Click on your new deployment (called `ML Flow Deployment`). Add a schedule by clicking "Add" and choosing your desired frequency.

<img src="img/adding-a-schedule.png" style="width: 900px;"/>

## Execute flow runs from this deployment 
Starting a local agent will enable you to run execute the code you created in the last step, as frequently as you specified in your schedule. To do this, you simply need to:
1. Copy this command: `prefect agent start -q default`.
2. Paste in a new terminal window: Visit the `Jupyter Home Page` tab that is already open in your browser. Click "New" in the right-hand corner and choose "Terminal". Paste the command you copied and hit Enter. In the terminal, you should see that the agent picks up the flow run that you just created :)
3. Now when you look at the UI, in the Flow Runs tab you'll see the all of your scheduled flow runs for this deployment, and with the agent running, each flow will move from a `Scheduled` to `Completed` state as the execution time occurs.

## Have some fun
Experiment on your own in the UI by adding a description to your deployment, parameters, and much more!

While in this demo, we use a static dataset (titanic.csv), it's easy to see how the `create_data()` task within `prefect_ml.py` could be altered to draw from any number of sources, using the `requests` library, for instance, to request data from an API.

## Viewing or removing metadata
You can view metadata about your flows and deployments by cd-ing into your orion.db: `open ~/.prefect/orion.db`. This will open a SQLite browser if one exists.

Alternatively, you can `rm -rf ~/.prefect/orion.db` if you would like to remove all of your existing flows, flow runs, and deployments that you created. If you only want to remove one flow, flow run, or deployment, I would recommend doing so through the UI. 

## Happy engineering!
We hope you were able to learn a little more about how Prefect works. We've barely scratched the surface of what's possible with Prefect. For instance, you can also store repeated configuration with blocks and integrate with other tools easily. Please see our docs to learn even more about the possibilities Prefect can give your workflows:
https://docs.prefect.io/

Some things we didn't touch on in this tutorial:
- Other build commands that can be used during deployment: https://docs.prefect.io/concepts/deployments/#deployment-build-options
- Flow Storage: https://docs.prefect.io/concepts/storage/
- We can set log levels to 'debug', 'error', 'info', and more: https://docs.prefect.io/concepts/logs/
- Specifying infrastructure allows you to deploy your agents and control where your flows are run: https://docs.prefect.io/concepts/infrastructure/

<img src="img/marvin.png" style="width: 300px;"/>

Prefect's mascot, Marvin, is always cooking up new recipes on how to use Prefect. Stop on by [prefect-recipes](https://github.com/PrefectHQ/prefect-recipes) to see more!